In [ ]:
import json
import cv2
import os
from glob import glob
from sklearn.model_selection import train_test_split
import statistics
ann_path = './custom_dataset/train/*.txt'
ann_files = glob(ann_path)
# valid set을 나눌 경우
ann_train, ann_valid = train_test_split(ann_files,
                                       test_size=0.01,
                                       random_state=1119)
ratio = []
def labelmetxt2coco(anns):
    cars = {}
    # 숫자로 해도 되지만, 클래스 확인하기 위해 차종으로 변경
    classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", 
               "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", 
               "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", 
               "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", 
               "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", 
               "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", 
               "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
               "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", 
               "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
               "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", 
               "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", 
               "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", 
               "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", 
               "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", 
               "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", 
               "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", 
               "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]
    cars["categories"] = [{"id":i,"name":cat,"supercategory":"none"} for i, cat in enumerate(classes)]
    cars["images"] = []
    cars["annotations"] = []
    cnt_ann = 0
    for i, ann in enumerate(anns):
        img_path = ann.replace("txt", "png")
        #h, w, _ = cv2.imread(img_path).shape
        cars["images"].append({"id":i,"height":1040,"width":1920,"file_name":img_path})

        f_ann = open(ann, "r")
        for line in f_ann.readlines():
            data = line.split()
            cat = int(float(data[0]))
            pt1x = int(data[1])
            pt1y = int(data[2])
            #pt2x = int(data[3])    not to use
            #pt2y = int(data[4])
            pt3x = int(data[5])
            pt3y = int(data[6])
            #pt4x = int(data[7])
            #pt4y = int(data[8])
            x = pt1x
            y = pt1y
            width = pt3x - pt1x
            height = pt3y - pt1y
            area = width * height
            cars["annotations"].append({"id": cnt_ann,
                                        "image_id": i,
                                        "category_id": cat,
                                        "bbox": [x, y, width, height],
                                        "area": area,
                                        "segmentation": [],
                                        "iscrowd": 0})
            ratio.append(width / height)
            cnt_ann += 1 
    return cars

labelmetxt2coco(ann_train)
print(min(ratio), max(ratio), statistics.mean(ratio))

# with open('/content/drive/MyDrive/DACON/236107/data/annotations/train_json', "w") as f:
#     json.dump(labelmetxt2coco(ann_files), f, ensure_ascii=False, indent=4)

# train/valid split을 했을 경우
# with open('./custom_dataset/annotations/train.json', "w") as f:
#     json.dump(labelmetxt2coco(ann_train), f, ensure_ascii=False, indent=4)

# with open('./custom_dataset/annotations/val.json', "w") as f:
#     json.dump(labelmetxt2coco(ann_valid), f, ensure_ascii=False, indent=4)

# def labelmetxt2cocotest(anns):
#     cars = {}
#     # 숫자로 해도 되지만, 클래스 확인하기 위해 차종으로 변경
#     classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", 
#                "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", 
#                "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", 
#                "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", 
#                "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", 
#                "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", 
#                "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
#                "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", 
#                "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
#                "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", 
#                "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", 
#                "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", 
#                "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", 
#                "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", 
#                "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", 
#                "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", 
#                "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]
#     cars["categories"] = [{"id":i,"name":cat,"supercategory":"none"} for i, cat in enumerate(classes)]
#     cars["images"] = []
#     cars["annotations"] = []
#     cnt_ann = 0
#     for i, ann in enumerate(anns):
#         cars["images"].append({"id":i,"height":1040,"width":1920,"file_name":ann})
#         cars["annotations"].append({"id": cnt_ann,
#                                     "image_id": i,
#                                     "category_id": 0,
#                                     "bbox": [0, 0, 0, 0],
#                                     "area": 0,
#                                     "segmentation": [],
#                                     "iscrowd": 0})
#         cnt_ann += 1
        
#     return cars
# ann_path = './custom_dataset/test/*.png'
# ann_test = glob(ann_path)

# with open('./custom_dataset/annotations/test.json', "w") as f:
#     json.dump(labelmetxt2cocotest(ann_test), f, ensure_ascii=False, indent=4)

In [ ]:
!python tools/train.py configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_1x_cars.py --seed 41 --deterministic

In [ ]:
!python tools/train.py configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_1x_cars.py --seed 41 --deterministic

In [ ]:
!python tools/train.py configs/libra_rcnn/libra_faster_rcnn_x101_64x4d_fpn_1x_coco.py --seed 41 --deterministic

In [ ]:
!python tools/test.py work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_cars/cascade_rcnn_x101_64x4d_fpn_1x_cars.py work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_cars/epoch_20.pth --out work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_cars/result.pkl --format-only

In [ ]:
!python tools/test.py work_dirs/libra_faster_rcnn_x101_64x4d_fpn_1x_coco/libra_faster_rcnn_x101_64x4d_fpn_1x_coco.py work_dirs/libra_faster_rcnn_x101_64x4d_fpn_1x_coco/epoch_20.pth --out work_dirs/libra_faster_rcnn_x101_64x4d_fpn_1x_coco/result.pkl --format-only

In [ ]:
!python tools/test.py work_dirs/libra_faster_rcnn_x101_64x4d_fpn_1x_cars/libra_faster_rcnn_x101_64x4d_fpn_1x_cars.py work_dirs/libra_faster_rcnn_x101_64x4d_fpn_1x_cars/epoch_24.pth --out work_dirs/libra_faster_rcnn_x101_64x4d_fpn_1x_coco/result.pkl --format-only

In [17]:
import pickle
import pandas as pd
import glob
from tqdm.auto import tqdm
with open("./work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_cars/result.pkl","rb") as fr:
    data = pickle.load(fr)
                                             
test_img_paths = sorted(glob.glob('./custom_dataset/test/*.png'))                                                                                                              

results = pd.read_csv('./custom_dataset/sample_submission.csv')                 
for idx in tqdm(range(len(test_img_paths))):                                                                            
  prediction = data[idx]                 
  #img = cv2.imread(test_img_paths[idx])
  filename = test_img_paths[idx].split('/')[-1].split('\\')[-1]             
  for i in range(34):
    if prediction[i].shape[0]!=0:               
      label = i                    
      for j in range(prediction[i].shape[0]):
        x1, y1, x2, y2 = prediction[i][j][0], prediction[i][j][1], prediction[i][j][2],prediction[i][j][3]
        score = prediction[i][j][4]                
        
        results = results.append({
            "file_name" : filename,               
            "class_id" : label,                                                        
            "confidence" : score,                     
            "point1_x" : x1, "point1_y" : y1,
            "point2_x" : x2, "point2_y" : y1,
            "point3_x" : x2, "point3_y" : y2,
            "point4_x" : x1, "point4_y" : y2,
        }, ignore_index = True)

results.to_csv('submit.csv', index = False)
print(len(results))

  0%|          | 0/3400 [00:00<?, ?it/s]

14506


In [16]:
import os
import cv2
import pandas as pd
from tqdm.auto import tqdm
pred = pd.read_csv('del_submit.csv')
img_path = './custom_dataset/test'
classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", 
               "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", 
               "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", 
               "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", 
               "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", 
               "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", 
               "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
               "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", 
               "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
               "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", 
               "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", 
               "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", 
               "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", 
               "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", 
               "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", 
               "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", 
               "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]

for i in tqdm(range(len(pred))):
    scores = pred.iloc[i, :]
    label = "{}".format(classes[int(scores[1])]) + ' ' + str(scores[2])
    # index = "y_dis:{} x_dis:{} dia:{}".format(int(scores[11]), int(scores[12]), int(scores[13]))
    image_path = os.getcwd()+'/custom_dataset/test/'+scores[0]
    image = cv2.imread(image_path)
    cv2.rectangle(image, (int(scores[3]), int(scores[4])), (int(scores[7]), int(scores[8])), (0, 255, 0), 2)
    cv2.putText(image, label, (int(scores[3]), int(scores[4])-4), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        2 * 0.75, (0, 255, 0), 2, lineType=cv2.LINE_AA)
    # if int(scores[13]) > 120000:
    cv2.imshow("Cars", image)
    cv2.waitKey()
    cv2.destroyAllWindows()

  0%|          | 0/8654 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
import pandas as pd
import numpy as np
df = pd.read_csv('submit.csv')
print(len(df))
df.insert(11, 'y_dis', df.apply(lambda x: float(x['point3_y']) - float(x['point1_y']), axis=1))
df.insert(12, 'x_dis', df.apply(lambda x: float(x['point3_x']) - float(x['point1_x']), axis=1))
df.insert(13, 'diagonal_dis', df.apply(lambda x: (float(x['y_dis'])**2 + float(x['x_dis'])**2)*0.5, axis=1))

# 너무 긴 박스 지우기
df = df.drop(df[df.y_dis >= 410].index)

# 너무 긴 박스 지우기
# df = df.drop(df[df.y_dis >= 400].index)

# 너무 짧은 박스 지우기
df = df.drop(df[df.y_dis <= 200].index)

# 너무 긴 박스 지우기
df = df.drop(df[df.x_dis >= 410].index)

# 너무 짧은 박스 지우기
df = df.drop(df[df.x_dis <= 200].index)

# df = df.drop(df[df.confidence < 0.1].index)

print(len(df))
df = df.drop(['y_dis', 'x_dis', 'diagonal_dis'], axis=1)
df.to_csv('del_submit.csv', index = False)

14506
9088


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('Results/Mixup_distortion_20.csv')
print(len(df))
df.insert(11, 'y_dis', df.apply(lambda x: float(x['point3_y']) - float(x['point1_y']), axis=1))
df.insert(12, 'x_dis', df.apply(lambda x: float(x['point3_x']) - float(x['point1_x']), axis=1))
df.insert(13, 'diagonal_dis', df.apply(lambda x: (float(x['y_dis'])**2 + float(x['x_dis'])**2)*0.5, axis=1))

df.describe()

In [ ]:
#각 file_name에서 dis가 500이상인 애들을 잡고 그 x1 좌표에서 +- 75인 인덱스들 다 날리기
import pandas as pd
df = pd.read_csv('Results/Mixup_distortion_20.csv')
print(len(df))
df.insert(11, 'y_dis', df.apply(lambda x: float(x['point4_y']) - float(x['point1_y']), axis=1))
mask = df['y_dis'] >= 500

file_names = df.loc[mask, 'file_name']
point1_x_values = df.loc[mask, 'point1_x']

for file_name, point1_x in zip(file_names, point1_x_values):
    df = df.drop(df[(df['file_name'] == file_name) & (df['point1_x'] >= point1_x - 75) & (df['point1_x'] <= point1_x + 75)].index)
print(len(df))
df = df.drop(['y_dis'], axis=1)
df.to_csv('del_submit.csv', index = False)

In [19]:
from mmdet.apis import init_detector, inference_detector
import mmcv
from tqdm.auto import tqdm
import glob
import pandas as pd

# Specify the path to model config and checkpoint file
config_file = 'configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_1x_coco.py'
checkpoint_file = 'checkpoints/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth'
results = pd.read_csv('./custom_dataset/sample_submission.csv')
pred = pd.read_csv('final.csv')
img_path = './custom_dataset/test'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

for img in tqdm(glob.glob(img_path+'/*.png')):
    filename = img.split('/')[-1].split('\\')[-1]  
    iter_preds = pred.loc[pred['file_name'] == filename]
    result = inference_detector(model, img)
    bboxes = result[2]
    for bbox in bboxes:
        if bbox[4] < 0.5: continue
        X1, Y1, X2, Y2 = bbox[0], bbox[1], bbox[2],bbox[3]
        for iter in iter_preds.values:
            x1, y1, x2, y2 = iter[3], iter[4], iter[7], iter[8]
            if x1 <= X1 + 100 and x1 >= X1 - 100 and y1 <= Y1 + 100 and y1 >= Y1 - 100 and x2 <= X2 + 100 and x2 >= X2 - 100 and y2 <= Y2 + 100 and y2 >= Y2 - 100:
                results = results.append({
                "file_name" : iter[0],
                "class_id" : iter[1],
                "confidence" : iter[2],
                "point1_x" : x1, "point1_y" : y1,
                "point2_x" : x2, "point2_y" : y1,
                "point3_x" : x2, "point3_y" : y2,
                "point4_x" : x1, "point4_y" : y2,
            }, ignore_index = True)
        
print(len(results))
results.to_csv('Stacked_del_submit.csv', index = False) # 9138

c:\Python\lib\site-packages\mmcv\__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


load checkpoint from local path: checkpoints/faster_rcnn_x101_64x4d_fpn_1x_coco_20200204-833ee192.pth


  0%|          | 0/3400 [00:00<?, ?it/s]

c:\Users\KDY\Downloads\mmdetection-2.28.2\mmdet\datasets\utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)


KeyboardInterrupt: 

In [ ]:
import pickle
import pandas as pd
import glob

df = pd.read_csv('final.csv')                                                                                                           

results = pd.read_csv('./custom_dataset/sample_submission.csv')                 
for index, row in tqdm(df.iterrows()):                                                                            
    file_name = row['file_name'].split('.')[0] + '.png'
    confidence = row['confidence']
    classes = row['class_id']
    x1 = row['point1_x']
    y1 = row['point1_y']
    x2 = row['point3_x']
    y2 = row['point3_y']
    
    results = results.append({
        "file_name" : file_name,               
        "class_id" : classes,                                                        
        "confidence" : confidence,                     
        "point1_x" : x1, "point1_y" : y1,
        "point2_x" : x2, "point2_y" : y1,
        "point3_x" : x2, "point3_y" : y2,
        "point4_x" : x1, "point4_y" : y2,
    }, ignore_index = True)

results.to_csv('submit.csv', index = False)
print(len(results))